In [1]:
import pandas as pd
import pulp as pl
from scipy.optimize import linprog
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable
from fuzzywuzzy import process

In [2]:
routes = pd.read_csv('outputs/selected_routes.csv')
attractions = pd.read_csv('mr-qap/mr-qap-results.csv')

In [3]:
attractions = dict(zip(attractions.country, attractions.predicted_shares))

`min{sum(duration)}`

each `(conflict, crossing)` pair is a variable for total fatalities routed

the sum of `sum{(conflict, crossing)}` for each conflict `i` must equal total fatalities for that conflict.

`(conflict, crossing)` is continuous but can equal 0

the sum of each `country` fatalities divided by total fatalities must equal that country's `predicted_shares`

In [4]:
def get_fatalities(conflict):
    fatalities = routes[routes.conflict==conflict].iloc[0].fatalities
    return fatalities

def get_duration(conflict, variable):
    crossing = process.extractOne(variable.name, crossings)[0]
    duration = routes[(routes.conflict==conflict) & (routes.crossing==crossing)].iloc[0].duration
    return duration

def get_country(variable):
    country = process.extractOne(variable.name, countries)[0]
    return country

In [5]:
conflicts = routes.conflict.unique()
crossings = routes.crossing.unique()
countries = routes.crossing_country.unique()

In [6]:
model = LpProblem(name="refugee-routing", sense=LpMinimize)

In [7]:
variables = {}
for conf in conflicts:
    variables[conf] = []
    
for kk, vv in routes.iterrows():
    v = f"{vv.conflict}__{vv.crossing}__{vv.crossing_country}"
    x = LpVariable(name=v, lowBound=0, cat='Integer')
    variables[vv.conflict].append(x)

In [8]:
vars_by_country = {}
for country in countries:
    vars_by_country[country] = []

for kk, vv in variables.items():
    for i in vv:
        country = get_country(i)
        vars_by_country[country].append(i)

In [9]:
# start with only first 2 conflicts for simplicity
for conf in conflicts:
    model += (variables[conf][0] + \
              variables[conf][1] + \
              variables[conf][2] + \
              variables[conf][3] + \
              variables[conf][4] + \
              variables[conf][5] + \
              variables[conf][6] == get_fatalities(conf), 
              f"{conf}_fatalities_constraint") 

In [10]:
total_fatalities = 0
for c in conflicts:
    total_fatalities += get_fatalities(c)

In [11]:
for c in countries:
    country_vars = []
    for ix, conf in enumerate(conflicts):
        c_var = vars_by_country[c][ix] 
        country_vars.append(c_var)
        
    model += (lpSum(country_vars)/total_fatalities == attractions[c],
              f"{c}_attraction")

In [12]:
obj_func_array = []
for c in conflicts:
    for i in range(7):
        fatalities = variables[c][i]
        duration = get_duration(c,fatalities)
        obj_func_array.append(fatalities*duration)

In [13]:
model += lpSum(obj_func_array)

In [14]:
model

refugee-routing:
MINIMIZE
43006.0*Baryshivka__Chop_(Tysa)___Zakhon__Hungary + 41268.0*Baryshivka__Malyi_Bereznyi___Ubl'a__Slovakia + 21706.0*Baryshivka__Platonove___Hoianul_Nou__Moldova + 32566.0*Baryshivka__Porubne___Siret__Romania + 16752.0*Baryshivka__Senkivka___Novi_Yurkovychi__Russian_Federation + 11983.0*Baryshivka__Slavutych___Komaryn__Belarus + 27007.0*Baryshivka__Yahodyn___Dorohusk__Poland + 72941.0*Bilohorivka__Chop_(Tysa)___Zakhon__Hungary + 10102.0*Bilohorivka__Krasna_Talivka___Voloshino__Russian_Federation + 71203.0*Bilohorivka__Malyi_Bereznyi___Ubl'a__Slovakia + 49287.0*Bilohorivka__Platonove___Hoianul_Nou__Moldova + 62501.0*Bilohorivka__Porubne___Siret__Romania + 42486.0*Bilohorivka__Vilcha___Oleksandrivka__Belarus + 57105.0*Bilohorivka__Yahodyn___Dorohusk__Poland + 38625.0*Borodianka__Chop_(Tysa)___Zakhon__Hungary + 36886.0*Borodianka__Malyi_Bereznyi___Ubl'a__Slovakia + 17832.0*Borodianka__Mohyliv_Podilskyi___Otach__Moldova + 27716.0*Borodianka__Porubne___Siret__Romania

In [15]:
status = model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/brandonrose/opt/anaconda3/envs/b39/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/pp/vgfp1wf143q46m_8v6qbt3bw0000gn/T/2cbd759282544a6787d621c39c467e19-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/pp/vgfp1wf143q46m_8v6qbt3bw0000gn/T/2cbd759282544a6787d621c39c467e19-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 58 COLUMNS
At line 1669 RHS
At line 1723 BOUNDS
At line 2046 ENDATA
Problem MODEL has 53 rows, 322 columns and 644 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2.3428e+08 - 0.00 seconds
Cgl0004I processed model has 53 rows, 322 columns (322 integer (0 of which binary)) and 644 elements
Cbc0013I At root node, 11 cuts changed objective from 2.3428032e+08 to 2.3428032e+08 in 1 passes
Cbc0014I Cut generator 0 (Probin

In [16]:
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")


for var in model.variables():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: -1, Infeasible
objective: 234280317.61100402
Baryshivka__Chop_(Tysa)___Zakhon__Hungary: 0.0
Baryshivka__Malyi_Bereznyi___Ubl'a__Slovakia: 0.0
Baryshivka__Platonove___Hoianul_Nou__Moldova: 0.0
Baryshivka__Porubne___Siret__Romania: 0.0
Baryshivka__Senkivka___Novi_Yurkovychi__Russian_Federation: 0.0
Baryshivka__Slavutych___Komaryn__Belarus: 0.0
Baryshivka__Yahodyn___Dorohusk__Poland: 40.0
Bilohorivka__Chop_(Tysa)___Zakhon__Hungary: 308.32592
Bilohorivka__Krasna_Talivka___Voloshino__Russian_Federation: 0.0
Bilohorivka__Malyi_Bereznyi___Ubl'a__Slovakia: 0.0
Bilohorivka__Platonove___Hoianul_Nou__Moldova: 0.0
Bilohorivka__Porubne___Siret__Romania: 0.0
Bilohorivka__Vilcha___Oleksandrivka__Belarus: 0.0
Bilohorivka__Yahodyn___Dorohusk__Poland: 316.67408
Borodianka__Chop_(Tysa)___Zakhon__Hungary: 0.0
Borodianka__Malyi_Bereznyi___Ubl'a__Slovakia: 0.0
Borodianka__Mohyliv_Podilskyi___Otach__Moldova: 0.0
Borodianka__Porubne___Siret__Romania: 0.0
Borodianka__Senkivka___Novi_Yurkovychi__Russian

In [17]:
refugee_counts = {}
for c in countries:
    refugee_counts[c] = 0

In [18]:
for var in model.variables():
    for c in countries:
        if c.split(' ')[0].lower() in var.name.lower():
            refugee_counts[c] += var.value()

In [19]:
refugee_counts

{'Poland': 993.389388,
 'Moldova': 648.7993389999999,
 'Romania': 916.111185,
 'Slovakia': 1081.005972,
 'Hungary': 581.724596,
 'Belarus': 137.278718,
 'Russian Federation': 1037.6908}

In [20]:
for kk, vv in refugee_counts.items():
    ratio = vv/total_fatalities
    print(f"{kk}: {ratio}")

Poland: 0.18409736619718312
Moldova: 0.12023709025203853
Romania: 0.1697759794292068
Slovakia: 0.20033468717568567
Hungary: 0.10780663380281691
Belarus: 0.025440829873980726
Russian Federation: 0.1923074128984433


In [21]:
attractions

{'Hungary': 0.107806634654595,
 'Belarus': 0.0254408299070104,
 'Moldova': 0.120237090184471,
 'Poland': 0.184097365747316,
 'Romania': 0.169775979722219,
 'Russian Federation': 0.192307412587041,
 'Slovakia': 0.200334687197347}